In [1]:
# 📦 Library Utama untuk Machine Learning & Deep Learning
import numpy as np                      # Untuk manipulasi array
import pandas as pd                    # Untuk manipulasi data tabular
import math                            # Untuk operasi matematika dasar

# 📊 Visualisasi
import matplotlib.pyplot as plt        # Untuk membuat grafik
import seaborn as sns # PASTIKAN INI ADA DI BAGIAN LIBRARIES Paling Atas Notebookmu!
import pandas as pd # Tambahkan jika belum terdefinisi di cell ini

# 🧠 TensorFlow & Keras
import tensorflow as tf                # Framework deep learning utama
from tensorflow.keras import backend as K  # Untuk operasi backend custom
from keras.models import Sequential    # Untuk membuat model sekuensial
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D  # (opsional) untuk model hybrid CNN-LSTM
from keras.metrics import RootMeanSquaredError as rmse
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model  # Untuk visualisasi arsitektur model
from keras import optimizers                # Untuk kustomisasi optimizer

# 📁 Data Collection dari MetaTrader 5
import MetaTrader5 as mt5              # Untuk mengakses data harga dari MT5

# 📅 Tanggal dan Waktu
from datetime import datetime, timedelta  # Untuk range tanggal
import time                               # Untuk tracking durasi training

# ⚙️ Utility
from sklearn.model_selection import train_test_split  # Untuk membagi data train/test
from sklearn.preprocessing import MinMaxScaler        # Untuk normalisasi data
import tf2onnx                                        # Untuk konversi model ke format ONNX
from sys import argv                                  # Untuk akses argument command line (opsional)
import gc                                             # Untuk garbage collection (mengurangi memory leak)

# 🔧 Pengaturan Awal
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# Menginisialisasi koneksi ke MetaTrader5
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
    
# Show terminal info
terminal_info=mt5.terminal_info()
print(terminal_info)


TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=True, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=4885, maxbars=100000, codepage=0, ping_last=16468, community_balance=0.0, retransmission=0.21606440495139373, company='MetaQuotes Ltd.', name='MetaTrader 5', language='English', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\miqba\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\miqba\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')


In [6]:
# Define start and end dates for data collection
end_date = datetime.now()
# start_date = end_date - timedelta(days=180)
start_date = datetime(2023, 1, 1) # Mulai dari 1 Januari 2020

# Cek apakah tanggal mulai lebih awal dari tanggal akhir
print("data start date =",start_date)
print("data end date =",end_date)

data start date = 2023-01-01 00:00:00
data end date = 2025-06-02 20:42:22.865920


In [7]:
# Ambil data harga XAUUSD (Timeframe H1) dari MT5
xauusd_rates = mt5.copy_rates_range("XAUUSD", mt5.TIMEFRAME_M15, start_date, end_date)

In [8]:


# Bikin DataFrame dari data yang diambil
df = pd.DataFrame(xauusd_rates)
df['time'] = pd.to_datetime(df['time'], unit='s') # Ubah timestamp ke datetime
# df.set_index('time', inplace=True)

# Cek beberapa baris awal dan akhir DataFrame
print("\nDataFrame Head:")
print(df.head())
print("\nDataFrame Tail:")
print(df.tail())
print(f"\nDataFrame Shape: {df.shape}") # Liat berapa banyak data (rows, columns)


DataFrame Head:
                 time     open     high      low    close  tick_volume  \
0 2023-01-02 22:00:00  1827.53  1830.07  1825.97  1826.11          285   
1 2023-01-02 22:15:00  1826.10  1829.08  1823.52  1827.92          594   
2 2023-01-02 22:30:00  1827.93  1828.36  1826.91  1827.95          624   
3 2023-01-02 22:45:00  1827.80  1829.44  1827.73  1829.44          603   
4 2023-01-02 23:00:00  1829.44  1832.05  1828.83  1831.60          899   

   spread  real_volume  
0    1190            0  
1     140            0  
2     140            0  
3     100            0  
4     120            0  

DataFrame Tail:
                     time      open      high       low     close  \
57705 2025-06-02 12:30:00  3360.939  3363.611  3360.295  3361.767   
57706 2025-06-02 12:45:00  3361.800  3362.194  3354.727  3355.489   
57707 2025-06-02 13:00:00  3355.533  3357.697  3352.740  3356.418   
57708 2025-06-02 13:15:00  3356.460  3357.783  3351.955  3355.774   
57709 2025-06-02 13:30:00 

In [9]:
# Simpan data ke CSV
df.to_csv("XAUUSD_M15_DATA.csv", index=False)  